In [1]:
%load_ext autoreload
%autoreload 2

In [53]:
import os
import json

import pandas as pd

In [19]:
results.keys()
experiments = results['experiments']

algorithms = {}
for algo in results['algorithms_info']:
    algorithms[algo['name']] = algo

print(results.keys())

print(experiments[0].keys())

print(experiments[0]['coverage_dist'])
print(experiments[0]['results']['Majority'])

dict_keys(['n_sources', 'n_dataitems', 'n_distinct', 'algorithms_info', 'experiments'])
dict_keys(['coverage_dist', 'truth_dist', 'distinct_dist', 'spread_dist', 'optimal', 'n_claims', 'iteration_index', 'results'])
{'name': 'TruncExponential', 'lmbda': 0.5, 'flipped': True}
{'time': 0.038550333999999964, 'scores': [1.0, 200]}


In [134]:
def results_to_dataframe(exp_name, in_parts, postfix_list=None, folder=None, return_optional=False):
    
    rows = []
    index = 0
    
    n_parts = 7 if in_parts else 1
    if postfix_list is not None and len(postfix_list) != n_parts:
        raise ValueError("Length of postfix list must match number of parts")
        
    for i in range(n_parts):
        if postfix_list is not None:
            pf_list = ['', postfix_list[i]]
        else:
            pf_list = ['']

        for postfix in pf_list:
        
            name = f"{exp_name}-part{i}" if in_parts else exp_name
            if folder is None:
                path = os.path.join("results", name + postfix + ".json")
            else:
                path = os.path.join("results", folder, name + postfix + ".json")

            with open(path, "r") as f:
                results = json.load(f)
                
            dictionary = {
                'n_sources': results['n_sources'],
                'n_dataitems': results['n_dataitems'],
                'n_distinct': results['n_distinct'],
            }

            algorithms = {}
            for algo in results['algorithms_info']:
                algorithms[algo['name']] = algo

            for experiment in results['experiments']:
                for dist in ['coverage_dist', 'truth_dist', 'distinct_dist', 'spread_dist']:
                    prefix = ""
                    if 'flipped' in experiment[dist] and experiment[dist]['flipped']:
                        prefix += "F_"
                    if experiment[dist]['name'] == 'TruncExponential' and experiment[dist]['lmbda'] == 0.5:
                        prefix += "H_"
                    dictionary[dist] = prefix + experiment[dist]['name']

                dictionary['optimal_perc_score'] = experiment['optimal'][0]
                dictionary['optimal_score'] = experiment['optimal'][1]
                dictionary['n_claims'] = experiment['n_claims']
                dictionary['iteration_index'] = experiment['iteration_index'] if 'iteration_index' in experiment else -1

                for key, value in experiment['results'].items():
                    dictionary[f'{key}_time'] = value['time']
                    dictionary[f'{key}_score_perc'] = value['scores'][0]
                    dictionary[f'{key}_score'] = value['scores'][1]
                
                rows.append(pd.DataFrame.from_dict({index: dictionary}, orient='index'))
                index += 1
        
    if return_optional:
        return pd.concat(rows), algorithms
    
    return pd.concat(rows)

In [139]:
exp_name = "run3-200-200-20"
df = results_to_dataframe(exp_name, in_parts=True, folder=exp_name)
df.to_csv(os.path.join("results", f"{exp_name}.csv"), index=False)

In [140]:
df.shape

(2401, 23)